In [13]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
#from tensorflow.keras.utils import to_categorical

In [6]:
pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tensorflow


In [26]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# — FER-2013_sampled is right under the project root
SAMPLE_DIR = os.path.join(BASE_DIR, "FER-2013_sampled")

# — point to the two splits
TRAIN_DIR = os.path.join(SAMPLE_DIR, r'C:\Users\ASUS\Desktop\Digital\Emoji-Detection-System\FER-2013_sampled\FER-2013_sampled\train_balanced_7000')
TEST_DIR  = os.path.join(SAMPLE_DIR, r'C:\Users\ASUS\Desktop\Digital\Emoji-Detection-System\FER-2013_sampled\FER-2013_sampled\test_balanced_3000')

# sanity check
assert os.path.isdir(TRAIN_DIR), f"Train folder not found: {TRAIN_DIR}"
assert os.path.isdir(TEST_DIR),  f"Test folder not found:  {TEST_DIR}"

In [19]:
EMOTIONS = ['angry','disgust','fear','happy','sad','surprise','neutral']
IMG_SIZE  = (64, 64)

In [16]:
images, labels = [], []
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
for idx, emo in enumerate(EMOTIONS):
    emo_folder = os.path.join(TRAIN_DIR, emo)
    for fname in os.listdir(emo_folder):
        img_path = os.path.join(emo_folder, fname)
        img_bgr  = cv2.imread(img_path)
        if img_bgr is None:
            continue

        # face‐crop (if found), else full image
        gray   = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        faces  = face_cascade.detectMultiScale(gray, 1.3, 5)
        if len(faces) > 0:
            x,y,w,h = faces[0]
            face = img_bgr[y:y+h, x:x+w]
        else:
            face = img_bgr

        # bilateral filter → grayscale → resize → normalize
        filt   = cv2.bilateralFilter(face, d=9, sigmaColor=75, sigmaSpace=75)#filtering
        gray_f = cv2.cvtColor(filt, cv2.COLOR_BGR2GRAY)# turn Grayscale
        small  = cv2.resize(gray_f, IMG_SIZE).astype('float32') / 255.0#reize

        images.append(small)
        labels.append(idx)

images = np.expand_dims(np.array(images), -1)
labels = np.array(labels, dtype=int)  

In [28]:
labels = np.eye(len(EMOTIONS))[labels]#one hot enocde 0 thru 7 

In [31]:
mask    = np.zeros(face.shape[:2], np.uint8)
bgModel = np.zeros((1,65), np.float64)
fgModel = np.zeros((1,65), np.float64)
rect    = (5,5, face.shape[1]-10, face.shape[0]-10)
cv2.grabCut(face, mask, rect, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)
mask2    = np.where((mask==2)|(mask==0), 0, 1).astype('uint8')
face_fg  = face * mask2[:, :, np.newaxis]
#segmentation 

In [29]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.3,
    random_state=42, stratify=labels.argmax(axis=1)
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5,
    random_state=42, stratify=y_temp.argmax(axis=1)
)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape,   y_val.shape)
print("Test: ", X_test.shape,  y_test.shape)

Train: (4505, 64, 64, 1) (4505, 7)
Val:   (965, 64, 64, 1) (965, 7)
Test:  (966, 64, 64, 1) (966, 7)
